# EXERCISE 4 - CHALLENGED AMADEUS: Match searches with bookings

* Guide:
    * For every search in the searches file, find out whether the search ended up in a booking or not (using the info in the bookings file). For instance, search and booking origin and destination should match. 
    * For the bookings file, origin and destination are the columns dep_port and arr_port, respectively. 
    * Generate a CSV file with the search data, and an additional field, containing 1 if the search ended up in a booking, and 0 otherwise.

## Solution:

En primer lugar, importamos la librería "Pandas":

In [2]:
import pandas as pd

Comenzamos viendo cuantos registros no duplicados hay en el fichero "searches" y generando un fichero csv con todos estos registros:

In [3]:
#Leemos el fichero fijando el número de filas que importaremos al DF en cada iteración con el tamaño de los Chunks.
#Además, indicamos en la lectura que no prediga de que tipo es cada elemento, y que fuerce que todos los elementos sean tipo string.
size_sm = 1000000
s_find = pd.read_csv('./searches.csv.bz2', sep = "^", 
                     dtype = str, low_memory=False, chunksize = size_sm)
#Creamos un DF vacío donde vamos a ir añadiendo los resultados de los Chunks
sf_chunks = pd.DataFrame()

#Para cada iteración de tamaño definido en el 'chunksize', se añadirán los registros duplicados a un DF vacío y nos dirá
#cuantos no duplicados hay en cada iteración.
for i, s in enumerate(s_find):
    sf_chunks = sf_chunks.append(s)
    sf_chunks.drop_duplicates(inplace=True)
    print (i, len(sf_chunks)) 

#Generamos un fichero csv con los registros no duplicados sin imprimir los índices
sf_chunks.to_csv('./searches.no_dup.csv', sep = "^", index = False)

0 358999
1 359003
2 359003
3 359003
4 359003
5 359003
6 359003
7 359003
8 359003
9 359003
10 359003
11 359003
12 359003
13 359003
14 359003
15 359003
16 359003
17 359003
18 359003
19 359003
20 359004


Hacemos lo mismo para el fichero "bookings":

In [4]:
#Leemos el fichero fijando el número de filas que importaremos al DF en cada iteración con el tamaño de los Chunks.
#Además, indicamos en la lectura que no prediga de que tipo es cada elemento, y que fuerce que todos los elementos sean tipo string.
size_sm = 1000000
b_find = pd.read_csv('./bookings.csv.bz2', sep = "^", 
                     dtype = str, low_memory=False, chunksize = size_sm)
#Creamos un DF vacío donde vamos a ir añadiendo los resultados de los Chunks
bf_chunks = pd.DataFrame()

#Para cada iteración de tamaño definido en el 'chunksize', se añadirán los registros duplicados a un DF vacío y nos dirá
#cuantos no duplicados hay en cada iteración.
for i, b in enumerate(b_find):
    bf_chunks = bf_chunks.append(b)
    bf_chunks.drop_duplicates(inplace=True)
    print (i, len(bf_chunks)) 

#Generamos un fichero csv con los registros no duplicados sin imprimir los índices
bf_chunks.to_csv('./bookings.no_dup.csv', sep = "^", index = False)

0 1000000
1 1000000
2 1000000
3 1000000
4 1000000
5 1000003
6 1000003
7 1000003
8 1000003
9 1000003
10 1000003


Tras generar los ficheros con los no duplicados, comenzamos con el plan de acción:

1) Seleccionamos las columnas de interés del fichero "bookings" para reducir el DF a lo que necesitamos:

Leemos el fichero de registros no duplicados y mostramos los nombres de las columnas para ver con claridad que columnas hay y como vienen sus nombres:

In [3]:
b = pd.read_csv('./bookings.no_dup.csv', sep="^", low_memory=False)
b.columns

Index(['act_date           ', 'source', 'pos_ctry', 'pos_iata', 'pos_oid  ',
       'rloc          ', 'cre_date           ', 'duration', 'distance',
       'dep_port', 'dep_city', 'dep_ctry', 'arr_port', 'arr_city', 'arr_ctry',
       'lst_port', 'lst_city', 'lst_ctry', 'brd_port', 'brd_city', 'brd_ctry',
       'off_port', 'off_city', 'off_ctry', 'mkt_port', 'mkt_city', 'mkt_ctry',
       'intl', 'route          ', 'carrier', 'bkg_class', 'cab_class',
       'brd_time           ', 'off_time           ', 'pax', 'year', 'month',
       'oid      '],
      dtype='object')

Como queremos saber la fecha en la que se hizo la reserva para poder matchearla con la fecha de la búsqueda, seleccionamos como columnas de interés: origen del vuelo, destino del vuelo y fecha de la reserva.

In [4]:
b = pd.read_csv('bookings.no_dup.csv', sep="^", 
              usecols=['dep_port', 'arr_port', 'cre_date           '], low_memory=False)

2) Quitamos los espacios en blanco de los elementos del DF del fichero de reservas:

In [15]:
b.columns=b.columns.str.strip()
b.dep_port=b.dep_port.str.strip()
b.arr_port=b.arr_port.str.strip()

3) Vemos en que formatos están las fechas de ambos ficheros y nivelamos sus formatos a "str":

Necesitamos leer el fichero "searches" sin seleccionar las columnas interesantes ya que queremos sacar toda la información de las búsquedas que han terminado en reservas:

In [6]:
s = pd.read_csv('searches.no_dup.csv', sep="^", low_memory=False)

In [49]:
#Fecha de la búsqueda del vuelo
s.Date[6]

'2013-01-01'

In [50]:
#Fecha de la reserva del vuelo
b.cre_date[6]

'2013-03-20'

Vemos que las fechas tienen distintos formatos, por lo que a la fecha de reserva del vuelo 'cre_date' le cambio el formato a 'str':

In [51]:
b.cre_date = b.cre_date.str[:10]
b.cre_date[6]

'2013-03-20'

4) Creamos una columna con 1's al fichero de reservas, para que al hacer el merge de "bookings" con "searches" los registros con reserva contengan un 1 en esta columna:

In [19]:
b['flight bookings'] = 1

In [11]:
b.head()

,cre_date,dep_port,arr_port,flight bookings
0,2013-02-22,ZRH,LHR,1
1,2013-03-26,SAL,CLT,1
2,2013-03-26,SAL,CLT,1
3,2013-03-26,AKL,SVO,1
4,2013-03-26,AKL,SVO,1


5) Hacemos un merge para obtener todas las búsquedas que han terminado en reserva.

In [43]:
s_b = s.merge(b,
           how ='left',
           left_on = ['Date', 'Origin', 'Destination'],
           right_on = ['cre_date', 'dep_port', 'arr_port'])

Eliminamos las columnas que hemos añadido en el merge del fichero "bookings", ya que una vez hecho ya no son útiles:

In [44]:
s_b.drop(['cre_date', 'dep_port', 'arr_port'], axis = 1, inplace = True)

6) Asignamos a las búsquedas que no han terminado en reserva y que por tanto tienen el valor 'NaN' en la columna 'flight bookings', el valor '0':

In [45]:
s_b['flight bookings'] = s_b['flight bookings'].fillna(0)
s_b.head()

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,...,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice,flight bookings
0,2013-01-01,20:25:57,MPT,624d8c3ac0b3a7ca03e3c167e0f48327,DE,TXL,AUH,1.0,2.0,TXL,...,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,FRA,0.0
1,2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5cf2986e8e02,MD,ATH,MIL,0.0,1.0,ATH,...,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,KIV,0.0
2,2013-01-01,18:04:49,MPT,3561a60621de06ab1badc8ca55699ef3,US,ICT,SFO,1.0,2.0,ICT,...,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,NYC,0.0
3,2013-01-01,17:42:40,FXP,1864e5e8013d9414150e91d26b6a558b,SE,RNB,ARN,0.0,1.0,RNB,...,NaN,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,STO,0.0
4,2013-01-01,17:48:29,MPT,1ec336348f44207d2e0027dc3a68c118,NO,OSL,MAD,1.0,2.0,OSL,...,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,OSL,0.0


Como el "fillna(0)" me da el valor en formato 'float', cambio el formato de la columna a 'integer':

In [48]:
s_b = s_b.astype({'flight bookings':int})
s_b.head()

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,...,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice,flight bookings
0,2013-01-01,20:25:57,MPT,624d8c3ac0b3a7ca03e3c167e0f48327,DE,TXL,AUH,1.0,2.0,TXL,...,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,FRA,0
1,2013-01-01,10:15:33,MPT,b0af35b31588dc4ab06d5cf2986e8e02,MD,ATH,MIL,0.0,1.0,ATH,...,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,KIV,0
2,2013-01-01,18:04:49,MPT,3561a60621de06ab1badc8ca55699ef3,US,ICT,SFO,1.0,2.0,ICT,...,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,NYC,0
3,2013-01-01,17:42:40,FXP,1864e5e8013d9414150e91d26b6a558b,SE,RNB,ARN,0.0,1.0,RNB,...,NaN,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,STO,0
4,2013-01-01,17:48:29,MPT,1ec336348f44207d2e0027dc3a68c118,NO,OSL,MAD,1.0,2.0,OSL,...,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,OSL,0


7) Por útlimo, escribimos las búsquedas de vuelos realizas que han acabado en reservas, añadiéndole una columna con el valor '1' si la búsqueda se ha convertido en vuelo, y con el valor '0' si no ha sido así, a un fichero csv:

In [47]:
s_b.to_csv('./searches_with_bookings.csv', sep = "^", index = False)